In [7]:
#pip install pandas as pd
!pip install yfinance pandas

import yfinance as yf
import pandas as pd


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [39]:
FX_TICKERS = {
    "USDINR": "INR=X",
    "EURINR": "EURINR=X",
    "GBPINR": "GBPINR=X",
    "JPYINR": "JPYINR=X",
}

In [45]:
def fetch_fx_timeseries(
    pairs=("USDINR"),
    start="2020-01-01",   # 🔹 Changed to 2020
    end=None,
    interval="1d",
    auto_adjust=False,
):
    """
    Fetch FX OHLCV time series from Yahoo Finance via yfinance.

    Returns a tidy DataFrame:
    date, pair, open, high, low, close, adj_close, volume
    """
    tickers = []
    pair_for_ticker = {}
    print(pairs)

    for p in pairs:
        print(p)
        if p not in FX_TICKERS:
            raise ValueError(f"Unknown pair '{p}'. Available: {list(FX_TICKERS.keys())}")
        t = FX_TICKERS[p]
        tickers.append(t)
        pair_for_ticker[t] = p

    df = yf.download(
        tickers=" ".join(tickers),
        start=start,
        end=end,
        interval=interval,
        group_by="ticker",
        auto_adjust=auto_adjust,
        threads=True,
        progress=False,
    )

    if df is None or df.empty:
        raise RuntimeError("No data returned. Check tickers/dates/interval.")

    tidy_frames = []

    if isinstance(df.columns, pd.MultiIndex):
        for t in tickers:
            if t not in df.columns.get_level_values(0):
                continue
            sub = df[t].copy()
            sub.columns = [c.lower().replace(" ", "_") for c in sub.columns]
            sub = sub.reset_index()
            sub.insert(1, "pair", pair_for_ticker[t])
            tidy_frames.append(sub)
    else:
        sub = df.copy()
        sub.columns = [c.lower().replace(" ", "_") for c in sub.columns]
        sub = sub.reset_index()
        sub.insert(1, "pair", pair_for_ticker[tickers[0]])
        tidy_frames.append(sub)

    out = pd.concat(tidy_frames, ignore_index=True)
    print(out)

    expected_cols = ["Date", "pair", "open", "high", "low", "close", "adj_close", "volume"]
    for c in expected_cols:
        if c not in out.columns:
            out[c] = pd.NA

    out = out[expected_cols].sort_values(["pair", "date"]).reset_index(drop=True)
    return out


In [46]:
if __name__ == "__main__":
    fx_df = fetch_fx_timeseries(
        pairs=("USDINR"),
        start="2020-01-01",   # 🔹 From 2020 onwards
        interval="1d",
    )

    #print(fx_df.head())
    #print(fx_df.tail())

    #fx_df.to_csv("fx_yahoo_timeseries_from_2020.csv", index=False)
    #print("Saved to fx_yahoo_timeseries_from_2020.csv")

USDINR
U


ValueError: Unknown pair 'U'. Available: ['USDINR', 'EURINR', 'GBPINR', 'JPYINR']